<a href="https://colab.research.google.com/github/cbeuter/Disc_CSL_e_EB/blob/main/CSL_Estabilidade_e_Routh_Hurwitz_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estabilidade em Sistemas de Controle

A principal regra de Routh-Hurwitz é um método utilizado na teoria de controle e análise de sistemas dinâmicos para determinar a estabilidade de um sistema de controle linear. A regra estabelece que todos os coeficientes de um polinômio característico devem ser positivos para que o sistema seja considerado estável. Se houver qualquer coeficiente negativo ou zero, então o sistema é instável. Essa regra é aplicada construindo uma tabela conhecida como tabela de Routh, na qual os coeficientes do polinômio característico são organizados e manipulados para determinar a estabilidade do sistema.

## A Regra

A regra de montagem dos coeficientes, também conhecida como a regra de construção da tabela de Routh, é um procedimento sistemático para organizar os coeficientes do polinômio característico de um sistema de controle linear. Aqui está um resumo do processo passo a passo:

1. Escreva o polinômio característico do sistema, por exemplo: $a_n s^n + a_{n-1} s^{n-1} + ... + a_1 s + a_0$.

2. Liste os coeficientes `a_i` do polinômio em uma tabela, começando pelos termos de maior grau e preenchendo os espaços vazios com zeros, se necessário.

3. Divida os coeficientes em duas linhas alternadas na tabela. Na primeira linha, insira os coeficientes com índices pares (começando com $a_0$), e na segunda linha, insira os coeficientes com índices ímpares (começando com $a_1$).

4. Calcule os elementos restantes da tabela usando os coeficientes nas duas primeiras linhas. Os elementos são calculados da seguinte maneira:

   - Primeira coluna: Copie os coeficientes $a_{n-1}$ e $a_{n-3}$ diretamente.
   
   - Para as colunas subsequentes, calcule os elementos da seguinte forma:
   
     $$
     b_1 = (a_{n-2}a_{n-1} - a_{n-3}a_n) / a_{n-1}
     $$

     $$
     b_2 = (a_{n-2}a_{n-1} - a_{n-3}a_n) / a_{n-1}
     $$
     
     e assim por diante, onde $b_1$, $b_2$, ... são os elementos das colunas subsequentes.

5. Se todos os elementos de uma coluna forem iguais a zero, então isso indica uma mudança de sinal nos coeficientes do polinômio característico, o que implica em uma raiz no semiplano direito do plano complexo, tornando o sistema instável.

6. A estabilidade do sistema é determinada contando o número de mudanças de sinal na primeira coluna da tabela. O número de mudanças de sinal deve ser igual ao número de raízes com parte real positiva do polinômio característico para que o sistema seja estável.

Seguindo esses passos, é possível aplicar a regra de montagem dos coeficientes para determinar a estabilidade de um sistema de controle linear utilizando a tabela de Routh.


## O Algoritmo
Aplicando a regra em algotimo.


In [16]:
def routh_table(coefficients):
    n = len(coefficients)
    table = [[0] * (n // 2 + n % 2) for _ in range(n)]

    # Fill in even coefficients in the first row
    table[0] = coefficients[::2] + [0] * (n // 2 - len(coefficients) // 2)
    # Fill in odd coefficients in the second row
    table[1] = coefficients[1::2] + [0] * (n // 2 - len(coefficients) // 2)

    # Fill in the rest of the table
    for i in range(2, n):
        for j in range(n // 2):
            if table[i - 1][0] == 0:
                table[i - 1][0] = 1e-10  # Small value to avoid division by zero
            term1 = table[i - 1][0] * table[i - 2][0]
            term2 = table[i - 2][j + 1] if j + 1 < len(table[i - 2]) else 0
            term3 = table[i - 1][j + 1] if j + 1 < len(table[i - 1]) else 0
            table[i][j] = (term1 * term2 - term3) / table[i - 1][0]

    return table

def count_sign_changes(row):
    count = 0
    for i in range(len(row) - 1):
        if row[i] * row[i + 1] < 0:
            count += 1
    return count

def check_stability(coefficients):
    table = routh_table(coefficients)
    first_column = [row[0] for row in table]
    sign_changes = count_sign_changes(first_column)

    if sign_changes == 0:
        print("O sistema é estável.")
    else:
        print(f"O sistema é instável, pois há {sign_changes} mudanças de sinal na primeira coluna.")

# Exemplo de utilização
coefficients = [1, 2, 4, 8, 1, 2]
check_stability(coefficients)


O sistema é instável, pois há 2 mudanças de sinal na primeira coluna.


Neste algoritmo, coefficients é uma lista contendo os coeficientes do polinômio característico em ordem decrescente de potências de s. A função routh_table monta a tabela de Routh e a função count_sign_changes conta o número de mudanças de sinal na primeira coluna da tabela. A função check_stability utiliza as duas funções anteriores para determinar se o sistema é estável ou instável.

In [17]:
import numpy as np

def calcular_raizes(coefficients):
    # Calcula as raízes do polinômio
    roots = np.roots(coefficients)
    return roots

# Coeficientes do polinômio
coefficients = [1, 2, 4, 8, 1, 2]
raizes = calcular_raizes(coefficients)
print("Raízes do polinômio:", raizes)


Raízes do polinômio: [-2.00000000e+00+0.j         -2.77555756e-17+1.93185165j
 -2.77555756e-17-1.93185165j  2.22044605e-16+0.51763809j
  2.22044605e-16-0.51763809j]


Com outros valores de exemplo


In [18]:
coefficients = [1, 0, 4, 0, 1]
raizes = calcular_raizes(coefficients)
print("Raízes do polinômio:", raizes)

Raízes do polinômio: [0.+1.93185165j 0.-1.93185165j 0.+0.51763809j 0.-0.51763809j]


Gerando coeficientes para experimentar os casos especiais de Routh

In [19]:
import numpy as np

# Raízes fornecidas
raizes = [-1, -2, -4, -4j, 4j, -3j, 3j, -1j, 1j, 6, 4, 1]  # estas são as raizes desejadas

# Calcula o polinômio a partir das raízes fornecidas
polinomio = np.poly(raizes)

print("O polinômio resultante é:")
print(np.poly1d(polinomio))

coefficients = np.poly1d(polinomio).coeffs
coefficients = coefficients.tolist() # para evitar ndarray na saida

print("Coeficientes do polinômio:")
print(coefficients) # polinômio caracteristico


O polinômio resultante é:
   12     11     10      9       8        7       6        5
1 x  - 4 x  - 3 x  - 36 x - 365 x + 1028 x + 771 x + 9252 x
              4        3       2
 + 2.801e+04 x - 1024 x - 768 x - 9216 x - 2.765e+04
Coeficientes do polinômio:
[1.0, -4.0, -3.0, -36.0, -365.0, 1028.0, 771.0, 9252.0, 28012.0, -1024.0, -768.0, -9216.0, -27648.0]


Aplicando a extração padrão de raizes

In [20]:
raizes = calcular_raizes(coefficients)
raizes = np.around(raizes, decimals=10) # para liminar valores muito pequenos da resposta
print("Raízes do polinômio:", raizes)

# ou outra forma de visualizar a resposta
raizes_sem_zeros_negativos = [str(raiz).replace('-0', '0') for raiz in raizes]
print("Raízes do polinômio:", raizes_sem_zeros_negativos)


Raízes do polinômio: [ 6.+0.j  4.+0.j -4.+0.j  0.+4.j  0.-4.j -0.+3.j -0.-3.j -2.+0.j  1.+0.j
 -0.+1.j -0.-1.j -1.+0.j]
Raízes do polinômio: ['(6+0j)', '(4+0j)', '(-4+0j)', '4j', '-4j', '(0+3j)', '(0-3j)', '(-2+0j)', '(1+0j)', '(0+1j)', '(0-1j)', '(-1+0j)']


No exemplo acima, de fato, o polinômio apresentado e suas raizes são compatíveis com o desejado.

Agora, aplicando os mesmos dados com a proposta inicial de algoritmo de Routh

In [21]:
# coefficients, serão os mesmos já calculados anteriormente
check_stability(coefficients)

O sistema é instável, pois há 6 mudanças de sinal na primeira coluna.


In [22]:
def routh_table(coefficients):
    n = len(coefficients)
    table = [[] for _ in range(n)]

    # Fill in even coefficients in the first row
    table[0] = [coefficients[i] for i in range(0, n, 2)]
    # Fill in odd coefficients in the second row
    table[1] = [coefficients[i] for i in range(1, n, 2)]

    # Fill in the rest of the table
    for i in range(2, n):
        for j in range(n // 2):
            if table[i - 1][0] == 0:
                table[i - 1][0] = 1e-10  # Small value to avoid division by zero
            term1 = table[i - 1][0] * table[i - 2][0]
            term2 = table[i - 2][j + 1] if j + 1 < len(table[i - 2]) else 0
            term3 = table[i - 1][j + 1] if j + 1 < len(table[i - 1]) else 0
            table[i].append((term1 * term2 - term3) / table[i - 1][0])

    return table

def format_number(num):
    # Formatando números grandes no formato de engenharia
    if abs(num) > 1e6 or (abs(num) < 1e-4 and abs(num) > 0):
        return "{:.2e}".format(num)
    # Apresentando zero quando o número é muito pequeno
    elif abs(num) < 1e-10:
        return "0"
    else:
        return "{:.4f}".format(num)

def print_routh_table(table):
    for row in table:
        formatted_row = [format_number(val) for val in row]
        print("\t".join(formatted_row))

# Coeficientes do polinômio: 1s^3 + 3s^2 + 4s + 2
# coefficients = [1, 3, 4, 2]  # se comentado, usa os coef anteriores
table = routh_table(coefficients)

print("Tabela de Routh:")
print_routh_table(table)


Tabela de Routh:
1.0000	-3.0000	-365.0000	771.0000	28012.0000	-768.0000	-27648.0000
-4.0000	-36.0000	1028.0000	9252.0000	-1024.0000	-9216.0000
-12.0000	-108.0000	3084.0000	27756.0000	-3072.0000	-27648.0000
135.0000	-3855.0000	-34695.0000	3840.0000	34560.0000	0
1324.5556	-36751.0000	-333100.4444	36608.0000	331776.0000	0
-520397.2541	-4.68e+06	518372.3621	4.67e+06	0	0
-4.87e+07	-4.41e+08	4.85e+07	4.39e+08	0	0
2.44e+12	-2.70e+11	-2.43e+12	0	0	0
2.15e+16	-2.36e+15	-2.14e+16	0	0	0
-6.57e+23	-5.92e+24	0	0	0	0
-5.07e+31	-4.59e+32	0	0	0	0
3.89e+48	0	0	0	0	0
2.33e+64	0	0	0	0	0


Finalmente, avaliando onde estão os polos

In [15]:
def print_routh_table_with_root_counts(table):
    root_counts = sum(1 for val in table[0] if val > 0), sum(1 for val in table[0] if val < 0)
    for row in table:
        formatted_row = [format_number(val) for val in row]
        print("\t".join(formatted_row))
    print("\nNúmero de raízes à esquerda do eixo x:", root_counts[0])
    print("Número de raízes à direita do eixo x:", root_counts[1])

# Coeficientes do polinômio: 1s^3 + 3s^2 + 4s + 2
# coefficients = [1, 3, 4, 2]
table = routh_table(coefficients)

print("Tabela de Routh:")
print_routh_table_with_root_counts(table)


Tabela de Routh:
1.0000	-3.0000	-365.0000	771.0000	28012.0000	-768.0000	-27648.0000
-4.0000	-36.0000	1028.0000	9252.0000	-1024.0000	-9216.0000
-12.0000	-108.0000	3084.0000	27756.0000	-3072.0000	-27648.0000
135.0000	-3855.0000	-34695.0000	3840.0000	34560.0000	0
1324.5556	-36751.0000	-333100.4444	36608.0000	331776.0000	0
-520397.2541	-4.68e+06	518372.3621	4.67e+06	0	0
-4.87e+07	-4.41e+08	4.85e+07	4.39e+08	0	0
2.44e+12	-2.70e+11	-2.43e+12	0	0	0
2.15e+16	-2.36e+15	-2.14e+16	0	0	0
-6.57e+23	-5.92e+24	0	0	0	0
-5.07e+31	-4.59e+32	0	0	0	0
3.89e+48	0	0	0	0	0
2.33e+64	0	0	0	0	0

Número de raízes à esquerda do eixo x: 3
Número de raízes à direita do eixo x: 4


Conforme apresentado acima, o algotimo tem certa eficiência nos resultados. Porém, ao final, não apresenta uma boa informação, ou seja, quantos polos estão sobre o eixo $j\omega$

## Atividade 01
Apresente uma solução genérica o suficiente para o algoritmo acima gerar a tabela padrão de análise de Routh-Hurwitz, como exemplificado na imagem abaixo.


| Posição   | Par (4ªordem)   | Outro (4ª ordem)  | Total (8ª ordem) |
--|--|--|--
Semiplano da direita | 0 | 2 | 2 |
Semiplano da esquerda | 0 | 2 | 2 |
$j\omega$ | 4 | 0 | 4 |
Total | 4 | 4 | 8 |
